In [1]:
!pip install kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
# Install kaggle package
!pip install kaggle

# Import libraries
import sqlite3
import pandas as pd
import os

# Download Kaggle dataset (zygmunt/goodbooks-10k)
try:
    !kaggle datasets download -d zygmunt/goodbooks-10k
    !unzip -o goodbooks-10k.zip  # -o flag to overwrite without prompting
except Exception as e:
    print(f"Error downloading dataset: {e}")
    print("Ensure Kaggle API is configured and internet is active.")
    raise

# Connect to SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create tables with constraints (Database Concepts)
cursor.execute('''
CREATE TABLE Books (
    book_id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    authors TEXT,
    original_publication_year INTEGER,
    average_rating REAL,
    ratings_count INTEGER
)
''')

cursor.execute('''
CREATE TABLE BookTags (
    goodreads_book_id INTEGER,
    tag_id INTEGER,
    count INTEGER,
    FOREIGN KEY (goodreads_book_id) REFERENCES Books(book_id)
)
''')

# Load data into pandas
try:
    books_df = pd.read_csv('books.csv', encoding='latin1')
    book_tags_df = pd.read_csv('book_tags.csv', encoding='latin1')
    # Sample 10,000 rows to keep it fast (adjust if dataset is smaller)
    if len(books_df) > 10000:
        books_df = books_df.sample(n=10000, random_state=42)
    if len(book_tags_df) > 10000:
        book_tags_df = book_tags_df.sample(n=10000, random_state=42)
except FileNotFoundError:
    print("Error: books.csv or book_tags.csv not found. Check Kaggle download.")
    raise
except Exception as e:
    print(f"Error loading CSV file: {e}")
    raise

# Insert data into SQLite
books_df[['book_id', 'title', 'authors', 'original_publication_year', 'average_rating', 'ratings_count']].to_sql('Books', conn, if_exists='append', index=False)
book_tags_df[['goodreads_book_id', 'tag_id', 'count']].to_sql('BookTags', conn, if_exists='append', index=False)  # Fixed: removed redundant name

# Demonstrate INSERT (Basic SQL Syntax)
# Insert a new book manually
cursor.execute('''
INSERT INTO Books (book_id, title, authors, original_publication_year, average_rating, ratings_count)
VALUES (999999, 'Sample Book', 'John Doe', 2023, 4.5, 100)
''')

# Demonstrate UPDATE (Basic SQL Syntax)
# Update average_rating for older books (before 1950)
cursor.execute('''
UPDATE Books
SET average_rating = 3.0
WHERE original_publication_year < 1950 AND original_publication_year IS NOT NULL
''')

# Demonstrate DELETE (Basic SQL Syntax)
# Delete books with very few ratings
cursor.execute('''
DELETE FROM Books
WHERE ratings_count < 10
''')

# SQL Queries to demonstrate skills and provide insights
queries = [
    # Query 1: Most popular authors by ratings (Retrieve, Filter)
    '''
    SELECT
        authors,
        SUM(ratings_count) as total_ratings
    FROM Books
    WHERE authors IS NOT NULL
    GROUP BY authors
    ORDER BY total_ratings DESC
    LIMIT 5;
    ''',

    # Query 2: Books by publication decade (Retrieve, Manipulate)
    '''
    SELECT
        (original_publication_year / 10) * 10 as decade,
        COUNT(book_id) as book_count,
        SUM(ratings_count) as total_ratings
    FROM Books
    WHERE original_publication_year IS NOT NULL
    GROUP BY decade
    ORDER BY decade DESC
    LIMIT 5;
    ''',

    # Query 3: Authors with multiple highly rated books (Filter, Manipulate)
    '''
    SELECT
        authors,
        COUNT(book_id) as book_count
    FROM Books
    WHERE authors IS NOT NULL AND average_rating >= 4.0
    GROUP BY authors
    HAVING book_count > 5
    ORDER BY book_count DESC
    LIMIT 5;
    ''',

    # Query 4: Top books by genre popularity with subquery (Complex Query)
    '''
    SELECT
        b.title,
        b.authors,
        b.average_rating,
        b.ratings_count,
        bt.tag_id,
        bt.count as tag_count
    FROM Books b
    JOIN BookTags bt ON b.book_id = bt.goodreads_book_id
    WHERE b.ratings_count > (SELECT AVG(ratings_count) FROM Books)
    AND bt.count > (SELECT AVG(count) FROM BookTags)
    ORDER BY b.average_rating DESC, b.ratings_count DESC
    LIMIT 5;
    ''',

    # Query 5: Author consistency with CTE and window function (Complex Query)
    '''
    WITH AuthorRatings AS (
        SELECT
            authors,
            original_publication_year,
            average_rating,
            ROW_NUMBER() OVER (PARTITION BY authors ORDER BY average_rating DESC) as rating_rank
        FROM Books
        WHERE authors IS NOT NULL AND original_publication_year IS NOT NULL
    )
    SELECT
        ar.authors,
        COUNT(*) as total_books,
        AVG(ar.average_rating) as avg_rating,
        MAX(CASE WHEN rating_rank = 1 THEN average_rating END) as top_rating,
        SUM(CASE WHEN average_rating >= 4.0 THEN 1 ELSE 0 END) as highly_rated_count
    FROM AuthorRatings ar
    GROUP BY ar.authors
    HAVING total_books > 5
    ORDER BY avg_rating DESC
    LIMIT 5;
    ''',

    # Query 6: Cumulative ratings trend by decade with window function (Complex Query)
    '''
    SELECT
        decade,
        book_count,
        total_ratings,
        SUM(total_ratings) OVER (ORDER BY decade) as cumulative_ratings
    FROM (
        SELECT
            (original_publication_year / 10) * 10 as decade,
            COUNT(book_id) as book_count,
            SUM(ratings_count) as total_ratings
        FROM Books
        WHERE original_publication_year IS NOT NULL
        GROUP BY decade
    ) sub
    ORDER BY decade DESC
    LIMIT 5;
    '''
]

# Execute and display results
for i, query in enumerate(queries, 1):
    print(f"\nQuery {i} Results:")
    try:
        df_result = pd.read_sql_query(query, conn)
        print(df_result)
    except Exception as e:
        print(f"Error executing query {i}: {e}")

# Clean up
try:
    os.remove('goodbooks-10k.zip')
    os.remove('books.csv')
    os.remove('book_tags.csv')
except:
    pass
conn.close()

Dataset URL: https://www.kaggle.com/datasets/zygmunt/goodbooks-10k
License(s): CC-BY-SA-4.0
Archive:  goodbooks-10k.zip
  inflating: book_tags.csv           
  inflating: books.csv               
  inflating: ratings.csv             
  inflating: sample_book.xml         
  inflating: tags.csv                
  inflating: to_read.csv             

Query 1 Results:
                        authors  total_ratings
0  J.K. Rowling, Mary GrandPrÃ©       13372767
1               Suzanne Collins        8646393
2               Stephenie Meyer        8403438
3                  Stephen King        6505240
4                J.R.R. Tolkien        5262785

Query 2 Results:
   decade  book_count  total_ratings
0    2020           1            100
1    2010        3067      120581462
2    2000        3121      176212429
3    1990        1360       73491621
4    1980         704       34352837

Query 3 Results:
           authors  book_count
0     Nora Roberts          44
1        J.D. Robb          33
2